In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from keras.models import Model, load_model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout,GlobalAveragePooling1D,Conv1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 20000
maxlen = 100

def clean_text( text ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    #text = BeautifulSoup(review,'html.parser').get_text()
    #
    # 2. Remove non-letters
    text = re.sub("[^A-za-z0-9^,?!.\/'+-=]"," ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " _exclamationmark_ ", text)
    text = re.sub(r"\?", " _questionmark_ ", text)
    #
    return text

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").apply(clean_text).values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").apply(clean_text).values
print(y.shape)

Using TensorFlow backend.


(95851, 6)


In [2]:
list_sentences_train[:5]

array([ 'Nonsense _questionmark_   kiss off  geek  what I said is true   I will  have your account terminated ',
       '    Please do not vandalize pages  as you did with this edit to W  S  Merwin  If you continue to do so  you will be blocked from editing      ',
       '      Points of interest     I removed the   points of interest   section you added because it seemed kind of spammy  I know you probably did not  mean to disobey the rules  but generally  a point of interest tends to be rather touristy  and quite irrelevant to an area culture  That  just my opinion  though   If you want to reply  just put your reply here and add   talkback Jamiegraham08   on my talkpage     ',
       'Asking some his nationality is a Racial offence  Wow was not  aware of it   Blocking me has shown your support towards your community  Thanku for that',
       'The reader here is not going by my say so for ethereal vocal style and dark lyrical content  The cited sources in the External Links are sayin

In [3]:
print('test len',len(test))

test len 226998


In [4]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

(95851, 100) (226998, 100)


In [5]:
from sklearn.metrics import log_loss,accuracy_score

def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

def get_cnn_model():
    embed_size = 128
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Conv1D(64,
             3,
             padding='valid',
             activation='relu',
             strides=1)(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(32, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model
print('def model done')

def model done


In [6]:
from sklearn.model_selection import KFold
def kf_train(fold_cnt=3,rnd=1):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((95851,6)),np.zeros((226998,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = get_cnn_model()
        batch_size = 64
        epochs = 5
        file_path="weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), callbacks=callbacks_list)
        model = load_model(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    print('all eval',eval_val(y,train_pred))
    return train_pred, test_pred


train_pred,test_pred = kf_train()
print(train_pred.shape,test_pred.shape)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = test_pred
sample_submission.to_csv("../results/cnn_clean_1_csv.gz", index=False, compression='gzip')
import pickle
with open('../features/cnn_clean_1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)
print('done')

Train on 63900 samples, validate on 31951 samples
Epoch 1/5
63900/63900 [==============================] - 15s 235us/step - loss: 0.0924 - acc: 0.9715 - val_loss: 0.0537 - val_acc: 0.9807
Epoch 2/5
63900/63900 [==============================] - 12s 191us/step - loss: 0.0493 - acc: 0.9817 - val_loss: 0.0520 - val_acc: 0.9813
Epoch 3/5
63900/63900 [==============================] - 13s 196us/step - loss: 0.0407 - acc: 0.9844 - val_loss: 0.0538 - val_acc: 0.9812
Epoch 4/5
63900/63900 [==============================] - 12s 183us/step - loss: 0.0345 - acc: 0.9867 - val_loss: 0.0581 - val_acc: 0.9811
Epoch 5/5
63900/63900 [==============================] - 12s 184us/step - loss: 0.0291 - acc: 0.9887 - val_loss: 0.0612 - val_acc: 0.9805
Train on 63901 samples, validate on 31950 samples
Epoch 1/5
63901/63901 [==============================] - 12s 190us/step - loss: 0.0980 - acc: 0.9681 - val_loss: 0.0538 - val_acc: 0.9802
Epoch 2/5
63901/63901 [==============================] - 11s 178us/step 

In [7]:
train_pred[:10].round(3)

array([[ 0.047,  0.   ,  0.005,  0.001,  0.006,  0.002],
       [ 0.004,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.01 ,  0.   ,  0.001,  0.   ,  0.001,  0.   ],
       [ 0.001,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.193,  0.003,  0.028,  0.006,  0.042,  0.012],
       [ 0.005,  0.   ,  0.   ,  0.   ,  0.001,  0.   ],
       [ 0.04 ,  0.   ,  0.002,  0.   ,  0.007,  0.001],
       [ 0.003,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.016,  0.   ,  0.002,  0.   ,  0.002,  0.001]])

In [8]:
y[:10]

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [9]:
def get_nn_model():
    embed_size = 128
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model
print('def model done')

def model done


In [10]:
def kf_train(fold_cnt=3,rnd=1):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((95851,6)),np.zeros((226998,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = get_nn_model()
        batch_size = 64
        epochs = 6
        file_path="weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, 
                                     save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), callbacks=callbacks_list)
        model = load_model(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
        print('temp eval')
        eval_val(hold_out_y,hold_out_pred)
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    print('all eval',eval_val(y,train_pred))
    return train_pred, test_pred


train_pred,test_pred = kf_train()
print(train_pred.shape,test_pred.shape)

Train on 63900 samples, validate on 31951 samples
Epoch 1/6
63900/63900 [==============================] - 13s 197us/step - loss: 0.1291 - acc: 0.9652 - val_loss: 0.0744 - val_acc: 0.9754
Epoch 2/6
63900/63900 [==============================] - 12s 195us/step - loss: 0.0653 - acc: 0.9779 - val_loss: 0.0566 - val_acc: 0.9803
Epoch 3/6
63900/63900 [==============================] - 12s 184us/step - loss: 0.0530 - acc: 0.9810 - val_loss: 0.0532 - val_acc: 0.9812
Epoch 4/6
63900/63900 [==============================] - 12s 189us/step - loss: 0.0465 - acc: 0.9826 - val_loss: 0.0527 - val_acc: 0.9810
Epoch 5/6
63900/63900 [==============================] - 13s 201us/step - loss: 0.0424 - acc: 0.9839 - val_loss: 0.0526 - val_acc: 0.9814
Epoch 6/6
63900/63900 [==============================] - 13s 199us/step - loss: 0.0383 - acc: 0.9850 - val_loss: 0.0541 - val_acc: 0.9814
temp eval
0 0.11792112978 0.958999718319
1 0.0238699308046 0.990391537041
2 0.0559499751372 0.979155581985
3 0.01614124943

In [11]:
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = test_pred
sample_submission.to_csv("../results/nn_clean_1_csv.gz", index=False, compression='gzip')
import pickle
with open('../features/nn_clean_1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)
print('done')

done
